In [1]:
import warnings
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import (
    OneHotEncoder
)

from feature_engine.encoding import RareLabelEncoder
from sklearn.compose import ColumnTransformer
from feature_engine.datetime import DatetimeFeatures

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import eda_helper_functions
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.ensemble import IsolationForest
import sklearn

from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    MinMaxScaler,
    PowerTransformer,
    FunctionTransformer
)

from feature_engine.outliers import Winsorizer
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.encoding import (
    RareLabelEncoder,
    MeanEncoder,
    CountFrequencyEncoder
)

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

import warnings
pd.set_option('display.max_columns',None)
sklearn.set_config(transform_output = "pandas")
warnings.filterwarnings("ignore")

In [2]:
dtoj_transformer = Pipeline(steps = [
    ("dt",DatetimeFeatures(features_to_extract = ['weekend'],yearfirst = True,format = "mixed"))
])

In [3]:
tuning_data = pd.read_csv("../data/val_data.csv")
X_val = tuning_data.drop(columns = ['price'])
y_val = tuning_data.price

X_val = X_val.assign(
    date = pd.to_datetime(X_val.rename(columns={'dtoj_year': 'year', 
                                     'dtoj_month': 'month', 
                                     'dtoj_day': 'day'})[['year', 'month', 'day']])
)
weekend = dtoj_transformer.fit_transform(X_val[['date']])
X_val = X_val.assign(is_weekend = weekend)
X_val.drop(columns = ['dtoj_year','dep_time_min'],inplace = True)

In [4]:
import sys
import os

# Add the parent directory (project root) to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.feature_utils import (
    is_north, find_part_of_month, part_of_day,
    make_month_object, remove_duration, have_info,
    duration_category
)
from utils.rbf import RBFPercentileSimilarity


In [5]:
import joblib

# Define the path to the column transformer
load_path = os.path.join("..", "artifacts", "column_transformer.joblib")

# Load the column transformer
column_transformer = joblib.load(load_path)

print("✅ column_transformer loaded successfully.")


✅ column_transformer loaded successfully.


In [6]:
X_val_transformed = column_transformer.transform(X_val)
X_val_transformed.head(5)

,tf1__airline,tf2__date_weekend,tf3__source_Other,tf3__source_banglore,tf3__source_delhi,tf3__source_kolkata,tf3__destination_Other,tf3__destination_banglore,tf3__destination_cochin,tf3__destination_delhi,tf3__source_is_north,tf3__destination_is_north,tf4__part_of_month,tf4__dtoj_day,tf5__part_of_day,tf5__dep_time_hour,tf6__pca0,tf6__dtoj_month,tf7__pca0,tf7__total_stops,tf7__is_direct_flight,tf8__duration_rbf_25,tf8__duration_rbf_50,tf8__duration_rbf_75,tf8__duration_cat,tf8__duration,tf9__additional_info_Other,tf9__additional_info_in-flight meal not included,tf9__additional_info_no info
0,1.005556,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1,0,-1.035975,1.000000,4,0.391304,-0.300968,-0.031323,2.710959,2,0,-0.361153,-0.100935,-0.082143,2.0,2.031084,0.0,0.0,1.0
1,0.184966,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1,0,-1.035975,1.000000,16,0.739130,-0.321764,-0.212830,-0.209998,1,0,-0.361153,-0.100935,-0.082143,1.0,-0.609718,0.0,0.0,1.0
2,1.005556,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,-0.669521,0.538462,20,0.956522,-0.259376,1.249597,0.089383,1,0,-0.361153,-0.100935,-0.082143,2.0,-0.181480,0.0,1.0,0.0
3,0.184966,1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0,0,-0.669521,0.653846,12,0.608696,0.697084,-0.031323,-1.527130,0,1,-0.361153,-0.100935,-0.082143,0.0,-0.956387,0.0,0.0,1.0
4,-1.234900,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,-1.035975,0.884615,16,0.826087,-0.321764,-0.212830,-1.612668,0,1,-0.361153,-0.100935,-0.082143,0.0,-1.078741,0.0,0.0,1.0


In [7]:
X_train_transformed = pd.read_csv("../transformed_data/X_train_transformed.csv")
X_test_transformed = pd.read_csv("../transformed_data/X_test_transformed.csv")

y_train = pd.read_csv("../data/train_data.csv").price
y_test = pd.read_csv("../data/test_data.csv").price

### Model Tuning

In [8]:
# Taking the XGBoost Model 

import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import joblib  # for saving model

def objective(trial):
    # Suggest hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "random_state": 42,
        "n_jobs": -1
    }

    # Model
    model = XGBRegressor(**param)

    # Fit on training data
    model.fit(X_train_transformed, y_train)

    # Predict on tuning set
    preds = model.predict(X_val_transformed)

    # Evaluation metric
    mse = mean_squared_error(y_val, preds)
    return mse  # Minimize MSE


In [9]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100) 

[I 2025-08-06 18:03:40,734] A new study created in memory with name: no-name-75047491-cae3-4134-b466-48448596c0c6


[I 2025-08-06 18:03:44,458] Trial 0 finished with value: 2450955.5 and parameters: {'n_estimators': 443, 'max_depth': 9, 'learning_rate': 0.2594077880570082, 'subsample': 0.8673545988981637, 'colsample_bytree': 0.5784213488119463, 'gamma': 2.5649918425541163, 'min_child_weight': 10, 'reg_alpha': 9.198877306722347, 'reg_lambda': 9.946381336223961}. Best is trial 0 with value: 2450955.5.
[I 2025-08-06 18:03:48,624] Trial 1 finished with value: 2607408.0 and parameters: {'n_estimators': 649, 'max_depth': 17, 'learning_rate': 0.11779264419750902, 'subsample': 0.7277983810234394, 'colsample_bytree': 0.8786215084159206, 'gamma': 4.988135255586491, 'min_child_weight': 9, 'reg_alpha': 9.215259753978275, 'reg_lambda': 3.487634237857696}. Best is trial 0 with value: 2450955.5.
[I 2025-08-06 18:03:54,803] Trial 2 finished with value: 2536601.0 and parameters: {'n_estimators': 898, 'max_depth': 16, 'learning_rate': 0.07131541708008077, 'subsample': 0.9184688581610344, 'colsample_bytree': 0.6893439

In [10]:
best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

final_model = XGBRegressor(**best_params)
final_model.fit(X_train_transformed, y_train)

model_dir = os.path.join("..", "artifacts", "models")
os.makedirs(model_dir, exist_ok=True)

# Full save path for model
model_path = os.path.join(model_dir, "xgb_flight_price_model.joblib")

# Save the model
joblib.dump(final_model, model_path)

print(f"✅ Final model trained and saved as {model_path}")

✅ Final model trained and saved as ..\artifacts\models\xgb_flight_price_model.joblib


### Checking performance on Test data 

In [11]:
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error

y_pred = final_model.predict(X_test_transformed)

print("Test MSE:", mean_squared_error(y_test, y_pred))
print("Test MAE:", mean_absolute_error(y_test, y_pred))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("Test R²:", r2_score(y_test, y_pred))

Test MSE: 3209939.25
Test MAE: 864.950927734375
Test RMSE: 1791.6303329649227
Test R²: 0.8541452884674072


### Testing if the model is overfitting or not

In [12]:
# First on training data

y_pred = final_model.predict(X_train_transformed)

print("Train MSE:", mean_squared_error(y_train, y_pred))
print("Train MAE:", mean_absolute_error(y_train, y_pred))
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_pred)))
print("Train R²:", r2_score(y_train, y_pred))

Train MSE: 599858.0
Train MAE: 439.5081787109375
Train RMSE: 774.5050032117288
Train R²: 0.972586452960968


In [13]:
# On validation data

y_pred = final_model.predict(X_val_transformed)

print("Val MSE:", mean_squared_error(y_val, y_pred))
print("Val MAE:", mean_absolute_error(y_val, y_pred))
print("Val RMSE:", np.sqrt(mean_squared_error(y_val, y_pred)))
print("Val R²:", r2_score(y_val, y_pred))

Val MSE: 2007298.5
Val MAE: 776.6646728515625
Val RMSE: 1416.7916219402202
Val R²: 0.8922687768936157


In [14]:
# Taking the Random Forest model

import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def objective(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 5, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),  # ✅ fixed
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "random_state": 42,
        "n_jobs": -1
    }

    model = RandomForestRegressor(**param)
    model.fit(X_train_transformed, y_train)

    preds = model.predict(X_val_transformed)
    mse = mean_squared_error(y_val, preds)

    return mse

# Run the study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)


[I 2025-08-06 18:05:48,782] A new study created in memory with name: no-name-490615e7-bf33-47f6-b8e5-ef4c45cee2db
[I 2025-08-06 18:05:49,763] Trial 0 finished with value: 3560115.303328299 and parameters: {'n_estimators': 270, 'max_depth': 23, 'min_samples_split': 6, 'min_samples_leaf': 8, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 3560115.303328299.
[I 2025-08-06 18:05:51,821] Trial 1 finished with value: 3376573.94026061 and parameters: {'n_estimators': 836, 'max_depth': 15, 'min_samples_split': 4, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 1 with value: 3376573.94026061.
[I 2025-08-06 18:05:52,222] Trial 2 finished with value: 4026270.060576472 and parameters: {'n_estimators': 172, 'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': False}. Best is trial 1 with value: 3376573.94026061.
[I 2025-08-06 18:05:53,138] Trial 3 finished with value: 2865179.4041302116 and p

In [15]:
best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

final_rf_model = RandomForestRegressor(**best_params)
final_rf_model.fit(X_train_transformed, y_train)

model_dir = os.path.join("..", "artifacts", "models")
os.makedirs(model_dir, exist_ok=True)

# Full save path for model
model_path = os.path.join(model_dir, "randomforest_flight_price_model.joblib")

# Save the model
joblib.dump(final_rf_model, model_path)

print(f"✅ Final model trained and saved as {model_path}")

✅ Final model trained and saved as ..\artifacts\models\randomforest_flight_price_model.joblib


In [16]:
from sklearn.metrics import r2_score, mean_absolute_error

y_pred = final_rf_model.predict(X_test_transformed)

print("Test MSE:", mean_squared_error(y_test, y_pred))
print("Test MAE:", mean_absolute_error(y_test, y_pred))
print("Test R²:", r2_score(y_test, y_pred))

Test MSE: 3509807.060552969
Test MAE: 952.6452879383696
Test R²: 0.8405197911262379


In [17]:
y_pred = final_rf_model.predict(X_train_transformed)

print("Train MSE:", mean_squared_error(y_train, y_pred))
print("Train MAE:", mean_absolute_error(y_train, y_pred))
print("Train R²:", r2_score(y_train, y_pred))

Train MSE: 1437254.6608037022
Train MAE: 620.3354091566941
Train R²: 0.9343173980757004
